In [1]:
import pandas as pd
import numpy as np

In [18]:
COLUMNS_CHART_DATA = ['date', 'open', 'high', 'low', 'close', 'volume']

COLUMNS_TRAINING_DATA_V1 = [
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
]

COLUMNS_TRAINING_DATA_V1_RICH = [
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
    'inst_lastinst_ratio', 'frgn_lastfrgn_ratio',
    'inst_ma5_ratio', 'frgn_ma5_ratio',
    'inst_ma10_ratio', 'frgn_ma10_ratio',
    'inst_ma20_ratio', 'frgn_ma20_ratio',
    'inst_ma60_ratio', 'frgn_ma60_ratio',
    'inst_ma120_ratio', 'frgn_ma120_ratio',
]

COLUMNS_TRAINING_DATA_V2 = [
    'per', 'pbr', 'roe',
    'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio',
    'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'close_ma5_ratio', 'volume_ma5_ratio',
    'close_ma10_ratio', 'volume_ma10_ratio',
    'close_ma20_ratio', 'volume_ma20_ratio',
    'close_ma60_ratio', 'volume_ma60_ratio',
    'close_ma120_ratio', 'volume_ma120_ratio',
    'market_kospi_ma5_ratio', 'market_kospi_ma20_ratio', 
    'market_kospi_ma60_ratio', 'market_kospi_ma120_ratio', 
    'bond_k3y_ma5_ratio', 'bond_k3y_ma20_ratio', 
    'bond_k3y_ma60_ratio', 'bond_k3y_ma120_ratio'
]

In [19]:
def preprocess(data):
    windows = [5, 10, 20, 60, 120]
    for window in windows:
        data['close_ma{}'.format(window)] = \
            data['Close'].rolling(window).mean()
        data['volume_ma{}'.format(window)] = \
            data['Volume'].rolling(window).mean()
        data['close_ma%d_ratio' % window] = \
            (data['Close'] - data['close_ma%d' % window]) \
            / data['close_ma%d' % window]
        data['volume_ma%d_ratio' % window] = \
            (data['Volume'] - data['volume_ma%d' % window]) \
            / data['volume_ma%d' % window]

    data['open_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'open_lastclose_ratio'] = \
        (data['Open'][1:].values - data['Close'][:-1].values) \
        / data['Close'][:-1].values
    data['high_close_ratio'] = \
        (data['High'].values - data['Close'].values) \
        / data['Close'].values
    data['low_close_ratio'] = \
        (data['Low'].values - data['Close'].values) \
        / data['Close'].values
    data['close_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'close_lastclose_ratio'] = \
        (data['Close'][1:].values - data['Close'][:-1].values) \
        / data['Close'][:-1].values
    data['volume_lastvolume_ratio'] = np.zeros(len(data))
    data.loc[1:, 'volume_lastvolume_ratio'] = \
        (data['Volume'][1:].values - data['Volume'][:-1].values) \
        / data['Volume'][:-1] \
            .replace(to_replace=0, method='ffill') \
            .replace(to_replace=0, method='bfill').values

    return data

## V2 csv파일 생성

In [24]:
kospi20=['LG전자','이마트','기아차','SK하이닉스','한국전력',\
'현대백화점','셀트리온','KB금융','LG화학','아모레퍼시픽','기업은행','현대모비스','대우조선해양','GS건설',\
'S-Oil','삼성전자','현대차','대한항공'] #'SK텔레콤','CJ제일제당'

for kospi in kospi20:
    v1 = pd.read_csv("../../kospi_V1/data/V1/{}_V1.csv".format(kospi))
    del v1['Unnamed: 0']
    v2 = preprocess(v1)
    v2.to_csv("../../kospi_V2/data/{}_V2.csv".format(kospi), index=False)

## V2 + 